In [ ]:
from sklearn.metrics import mean_absolute_percentage_error
import statsmodels.api as sm
import pandas as pd
import matplotlib.pyplot as plt
import ipyleaflet
from ipywidgets import IntSlider, Play, jslink, Layout
import json
import branca.colormap as cm
from sklearn.linear_model import LinearRegression

In [ ]:
def date_time(row):
    return str(row['DATE'] + ' ' + row['TIME'])

def sarima_model_data(csv_file, month, road, p, d, q, P, D, Q,):
    # Processing the data
    full_date_range = pd.date_range(start=f'2022-01-01', end=f'2022-12-31 23:00:00', freq='h')
    month_range = pd.date_range(start=f'2022-{month}-01', end=f'2022-{month}-30 23:00:00', freq='h')
    df = pd.read_csv(csv_file, skiprows=18, header=0, usecols=[2, 13, 16, 17], thousands=',').dropna(how='all', ignore_index=True)
    df['Time'] = df.apply(date_time, axis=1)
    df['Time'] = pd.to_datetime(df['Time'], format='%m/%d/%Y %I:%M %p')
    df['Month'] = df['Time'].dt.month
    df.set_index(df['Time'], inplace=True)
    df = df.reindex(full_date_range)
    df['traffic'] = df['VOLUME'].interpolate(method='time')
    df.drop(df[df['Month'] != month].index, inplace=True)
    df = df.reindex(month_range)
    df['dow'] = df.index.dayofweek
    df.drop(columns=['DATE', 'TIME', 'VOLUME', 'Month'], inplace=True)
    
    # Splitting Data into Test and Train sets
    train_size = int(len(df) * 0.75)
    train, test = df.iloc[:train_size], df.iloc[train_size:]
        
    # Fitting the data to the SARIMA model
    sarima_model = sm.tsa.statespace.SARIMAX(train['traffic'], order=(p, d, q), seasonal_order=(P, D, Q, 24), enforce_stationarity=False, enforce_invertibility=False).fit()    
    
    # Preparing the data for Linear Regression
    residuals = sarima_model.resid
    regression_data = pd.concat([residuals, train['dow']], axis=1).dropna(how='all', ignore_index=True)

    X = regression_data.drop(columns=[0])
    y = regression_data[0]
    
    lineear_model = LinearRegression()
    lineear_model.fit(X, y)
    
    # Initial prediction using the SARIMA model (For Testing)
    test_pred = sarima_model.predict(start=test.index[0], end=test.index[-1], dynamic=False)
    mape_eval = mean_absolute_percentage_error(test['traffic'], test_pred)
        
    # Forecasting future traffic conditions
    hours_f = 168  # 1 Week
    forecast = sarima_model.get_forecast(steps=hours_f)
    forecast_index = pd.date_range(start=test.index[0], periods=hours_f, freq='h')
    
    # Prediction using the Linear Regression model
    exog_future = pd.DataFrame({'date': forecast_index, 'dow': forecast_index.dayofweek})
    exog_future.set_index('date', inplace=True)
    linear_adjustment = lineear_model.predict(exog_future)
    
    # Preparing and combining the data from both models
    traffic_forecast = forecast.predicted_mean
    df_f = traffic_forecast.to_frame()
    df_f['road_ref'] = road
    df_f['traffic'] = forecast.predicted_mean + linear_adjustment
    start_dt = df_f.index.min()
    end_dt = df_f.index.max()
    compare_traffic = df.loc[start_dt:end_dt]['traffic']
    
    # Evaluating the MAPE value for the combined model
    df_f['actual'] = compare_traffic # Adding the actual traffic data to the data frame for comparison
    combined_models_mape = mean_absolute_percentage_error(compare_traffic, df_f['traffic'])
    
    return df_f, combined_models_mape, mape_eval
# Setting the Month to July
month_value = 7

# Getting predictions for each road
d1_result = sarima_model_data('data/traffic_data/Location_D1_2022.csv', month_value, 'I 405', 3,0,0,3,1,2)
d12_results = sarima_model_data('data/traffic_data/Location_D12_2022.csv', month_value, 'SR 18', 3,0,0,3,1,2)
d13_results = sarima_model_data('data/traffic_data/Location_D13_2022.csv', month_value, 'WA 518', 3,0,0,3,1,2)
d14_results = sarima_model_data('data/traffic_data/Location_D14_2022.csv', month_value, 'WA 509', 2,0,0,3,1,2)
r046_results = sarima_model_data('data/traffic_data/Location_R046_2022.csv', month_value, 'I 5', 1,0,1,3,1,1)
s826_results = sarima_model_data('data/traffic_data/Location_S826_2022.csv', month_value, 'I 90', 3,0,0,3,1,2)

In [ ]:
# Variables to hold the data from the model
d1 = d1_result[0]
d1_mape = d1_result[1]

d12 = d12_results[0]
d12_mape = d12_results[1]

d13 = d13_results[0]
d13_mape = d13_results[1]

d14 = d14_results[0]
d14_mape = d14_results[1]

r046 = r046_results[0]
r046_mape = r046_results[1]

s826 = s826_results[0]
s826_mape = s826_results[1]

In [ ]:
# Defining the labels and data for the bar graph
roads = ['I 405', 'SR 18', 'WA 518', 'WA 509', 'I 5', 'I 90']
mape_values = [d1_mape, d12_mape, d13_mape, d14_mape, r046_mape, s826_mape]

# Plotting and displaying the bar graph
plt.figure(figsize=(10, 6))
plt.bar(roads, mape_values)
plt.xlabel('Roads')
plt.ylabel('MAPE (%)')
plt.title('MAPE (%) by Road')
plt.show()

In [ ]:
# Defining the color map for visually displaying the traffic volume on the map
step = cm.LinearColormap(['green', 'yellow', 'red'], vmin = d1['predicted_mean'].min(), vmax = d1['predicted_mean'].max(), caption='Traffic Volume', tick_labels=0)
# Defining the map and map controls
king_county_map = ipyleaflet.Map(center=(47.438620795789575, -121.79379682483055), zoom=10, layout=Layout(width='%80', height='800px'))
slider_1 = IntSlider(min=0, max=d1['road_ref'].count() - 1, step=1, value=0)
play = Play(value=0, min=0, max=d1['road_ref'].count() - 1, step=1, interval=200, repeat=False, autoplay=False)
jslink((play, 'value'), (slider_1, 'value'))

# Loading map data to visually see the roads along with the county the roads are located in
def json_load(geojson):
    with open(geojson, 'r') as f:
        data = json.load(f)
        return data

king_county_roads = ipyleaflet.GeoJSON(data=json_load('data/road_data/king_county.json'), style={'fillColor': 'black', 'fillOpacity': '.2'})
i_5 = ipyleaflet.GeoJSON(data=json_load('data/road_data/I_5.geojson'))
i_90 = ipyleaflet.GeoJSON(data=json_load('data/road_data/I_90.geojson'))
i_405 = ipyleaflet.GeoJSON(data=json_load('data/road_data/I_405.geojson'))
sr_18 = ipyleaflet.GeoJSON(data=json_load('data/road_data/SR_18.geojson'))
wa_509 = ipyleaflet.GeoJSON(data=json_load('data/road_data/WA_509.geojson'))
wa_518 = ipyleaflet.GeoJSON(data=json_load('data/road_data/WA_518.geojson'))

king_county_map.add_layer(i_5)
king_county_map.add_layer(i_90)
king_county_map.add_layer(i_405)
king_county_map.add_layer(sr_18)
king_county_map.add_layer(wa_509)
king_county_map.add_layer(wa_518)
king_county_map.add_layer(king_county_roads)

# Defining function to update map dynamically based on the slider control value
def update_map(change):
    i_5.style = {'color': step(r046.iloc[change.new]['predicted_mean'])}
    i_90.style = {'color': step(s826.iloc[change.new]['predicted_mean'])}
    i_405.style = {'color': step(d1.iloc[change.new]['predicted_mean'])}
    sr_18.style = {'color': step(d12.iloc[change.new]['predicted_mean'])}
    wa_509.style = {'color': step(d14.iloc[change.new]['predicted_mean'])}
    wa_518.style = {'color': step(d13.iloc[change.new]['predicted_mean'])}

slider_1.observe(update_map, names='value')

king_county_map.add_control(ipyleaflet.WidgetControl(widget=slider_1, position='bottomleft'))
king_county_map.add_control(ipyleaflet.WidgetControl(widget=play, position='bottomleft'))

display(king_county_map)

In [ ]:
# Defines and displays Line Plots for visually seeing the difference between the forecasted traffic and the actual traffic
fig, axs = plt.subplots(3,2)
fig.set_size_inches(20, 15)
axs[0,0].plot(d1.index, d1['actual'], label='Actual')
axs[0,0].plot(d1.index, d1['traffic'], label='Predicted')
axs[0,0].set_title('Traffic Volume Prediction Hourly: I 405')

axs[0,1].plot(d12.index, d12['actual'], label='Actual')
axs[0,1].plot(d12.index, d12['traffic'], label='Predicted')
axs[0,1].set_title('Traffic Volume Prediction Hourly: SR 18')

axs[1,0].plot(d13.index, d13['actual'], label='Actual')
axs[1,0].plot(d13.index, d13['traffic'], label='Predicted')
axs[1,0].set_title('Traffic Volume Prediction Hourly: WA 518')

axs[1,1].plot(d14.index, d14['actual'], label='Actual')
axs[1,1].plot(d14.index, d14['traffic'], label='Predicted')
axs[1,1].set_title('Traffic Volume Prediction Hourly: WA 509')

axs[2,0].plot(r046.index, r046['actual'], label='Actual')
axs[2,0].plot(r046.index, r046['traffic'], label='Predicted')
axs[2,0].set_title('Traffic Volume Prediction Hourly: I 50')

axs[2,1].plot(s826.index, s826['actual'], label='Actual')
axs[2,1].plot(s826.index, s826['traffic'], label='Predicted')
axs[2,1].set_title('Traffic Volume Prediction Hourly: I 90')